In [ ]:
import pandas as pd
import re
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

# Caminho do arquivo
processamento_output_txt = './processamento_executado.txt'

# Inicializar lista para armazenar os resultados
results = []

# Variáveis auxiliares
current_dataset = None
current_model = None

# Processar o arquivo
with open(processamento_output_txt, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        
        # Detectar nome do dataset e modelo
        if line.startswith("Arquivo:"):
            parts = line.split(" - ")
            current_dataset = parts[0].split(":")[1].strip()
            current_model = parts[1] if len(parts) > 1 else None

        # Detectar métricas
        elif "Mean Squared Error" in line:
            mse = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group()) if "Mean Squared Error" in line else None
        elif "Root Mean Squared Error" in line:
            rmse = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group()) if "Root Mean Squared Error" in line else None
        elif "Mean Absolute Error" in line:
            mae = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group()) if "Mean Absolute Error" in line else None
        elif "Mean Absolute Percentage Error" in line:
            mape = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group()) if "Mean Absolute Percentage Error" in line else None
        elif "R² Score" in line:
            r2 = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group())
        elif "Explained Variance Score" in line:
            explained_var = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group()) if "Explained Variance Score" in line else None
        elif "Max Error" in line:
            max_err = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group()) if "Max Error" in line else None
        
        # Adicionar resultados ao final do bloco
        elif line.startswith("--------------------------------------------------"):
            results.append({
                "Dataset": current_dataset,
                "Model": current_model,
                "MSE": mse if 'mse' in locals() else None,
                "RMSE": rmse if 'rmse' in locals() else None,
                "MAE": mae if 'mae' in locals() else None,
                "MAPE": mape if 'mape' in locals() else None,
                "R²": r2 if 'r2' in locals() else None,
                "Explained Variance": explained_var if 'explained_var' in locals() else None,
                "Max Error": max_err if 'max_err' in locals() else None,
            })

# Criar DataFrame
df_results = pd.DataFrame(results)

# Salvar os resultados em um arquivo CSV
df_results.to_csv('tabela_resultados_processamento.csv', index=False)

# Exibir os primeiros resultados diretamente no terminal
print(df_results.head(10))
